In [2]:
import numpy as np
import torch
eps = 1e-8

## Calculation Layers

### Multiplication Layer

In [1]:
class Mul :
    def __init__(self, W) :
        self.W = W
    
    def forward(self, X) :
        self.X = X
        return np.dot(self.X, self.W)

    def backward(self,dY) :
        X = self.X
        W = self.W
        dX = np.dot(dY, W.T)
        dY = np.dot(X.T, dY)
        return (dX,dY)

In [5]:
X = np.full((1,2),1)
W = np.full((2,1),2)

model = Mul(W)

y = model.forward(X)
print(y)
dx1,dx2 = model.backward(np.ones_like(y))
print(dx1)
print(dx2)

[[4]]
[[2 2]]
[[1]
 [1]]


In [6]:
x1 = torch.full((1,2),1.,requires_grad=True)
x2 = torch.full((2,1),2.,requires_grad=True)
y = torch.matmul(x1,x2)
print(y)
y.backward(torch.ones_like(y))
print(x1.grad)
print(x2.grad)

tensor([[4.]], grad_fn=<MmBackward0>)
tensor([[2., 2.]])
tensor([[1.],
        [1.]])


### Addition Layer

In [43]:
class Add :
    def __init__ (self,B) :
        self.B = B
    def forward(self,X) :
        return X+self.B
    def backward(self,dY) :
        db = np.sum(dY,axis=0)
        return (dY,db)

In [44]:
X = np.full((2,2),3)
b = np.full((1,),1)
print("X :\n",X,"\nb :\n",b)

model = Add(b)

y = model.forward(X)
print("y :\n",y)
dX,db = model.backward(np.ones_like(y))
print("dx :\n", dX)
print("db :\n", db)

X :
 [[3 3]
 [3 3]] 
b :
 [1]
y :
 [[4 4]
 [4 4]]
dx :
 [[1 1]
 [1 1]]
db :
 [2 2]


In [45]:
X = torch.full((2,2),3.,requires_grad=True)
b = torch.full((1,),1.,requires_grad=True)
print("X : ",X,"\nb : ",b )
y = X+b
print("y : \n",y)
y.backward(torch.ones_like(y))
print("dx :\n", X.grad)
print("db :\n", b.grad)

X :  tensor([[3., 3.],
        [3., 3.]], requires_grad=True) 
b :  tensor([1.], requires_grad=True)
y : 
 tensor([[4., 4.],
        [4., 4.]], grad_fn=<AddBackward0>)
dx :
 tensor([[1., 1.],
        [1., 1.]])
db :
 tensor([4.])


### Repetition Layer

In [28]:
class Repeat :
    def forward(self,X,N,axis) :
        self.axis = axis
        return np.repeat(X,N,axis=axis)
    def backward(self,dy) :
        return np.sum(dy, axis=self.axis, keepdims=True)

In [29]:
class Unsqueeze :
    def forward(self, X, axis) :
        self.axis = axis
        return np.expand_dims(X,axis = axis)
    def backward(self,dY) :
        return np.squeeze(dY,axis = self.axis)

In [30]:
class Squeeze :
    def forward(self,X,axis) :
        self.axis = axis
        return np.squeeze(X,axis=axis)
    def backward(self,dY) :
        return np.expand_dims(dY,axis=self.axis)

### SumLayer

In [ ]:
class Sum : 
    def forward(self,X,axis) :
        self.axis = axis
        self.N = X.shape[axis]
        return np.sum(X,axis=axis,keepdims=True)
    
    def backward(self,dy) :
        return np.repeat(dy,self.N,axis=self.axis)

## Activation Function Layers

In [ ]:
class ReLU:
    def forward(self,X) :
        Y = X.copy()
        self.mask = (X<0)
        Y[self.mask] = 0
        return Y
    def backward(self,dY) :
        dX = np.ones_like(dY)
        dX[self.mask] = 0.
        return dX

In [ ]:
class SigmoidLayer :
    def forward(self,X) :
        Y = 1./(1.+np.exp(-X))
        self.Y = Y
        return Y
    def backward(self,dY) :
        Y = self.Y
        return dY*Y*(1-Y)
        

In [ ]:
class SoftMax : 
    def forward(self,X) :
        c= np.max(X)
        exp_X = np.exp(X-c)
        sum_exp_X = np.sum(exp_X)
        Y = exp_X/sum_exp_X
        self.Y = Y
        return Y
    def backward(self,dY) :
        T = -dY*self.Y
        return self.Y-T
        
    def backward(self,dY,T) :
        return self.Y-T


## Loss Function Layers

In [ ]:
class BinaryEntropy :
    def forward(self,T,X) :
        self.T = T; self.X = X
        Y = -T*np.log(eps+X) - (1.-T)*np.log(eps+1.-X)
        return Y
    def backward(self,dY) :
        T = self.T
        X = self.X
        return -dY*(T/(X+eps) + (1.-T)/(eps+1.-X))

In [ ]:
class CrossEntropy :
    def __init__(self,T) :
        self.T = T  
    def forward(self,X) :
        self.X = X
        Y = - np.sum(self.T*np.log(eps+X),axis=1)
        return Y
    def backward(self,dY) :
        T = self.T; X = self.X
        return - dY*(T/X)   

## Utils

In [ ]:
def init_Xavier_weights(n,m) :
    W = np.sqrt(1./(n+m))*np.random.randn(n,m)
    b = np.zeros(m)
    return W,b

## Models

In [ ]:
class Linear :
    def __init__(self,W,b) :
        self.Mul = Mul(W)
        self.Add = Add(b)
    def forward(self,X) :
        Z = self.Mul.forward(X)
        Y = self.Add.forward(Z)
        return Y
        
    def backward(self,dY) :
        dZ,B = self.Add.backward(dY)
        db = np.sum(B,axis=0)
        dX,dW = self.Mul.backward(dZ)
        return dX, dW, db


In [ ]:
class CrossEntropyWithSoftMax :
    def __init__(self,T) :
        self.SoftMax = SoftMax()
        self.CrossEntropy = CrossEntropy(T)

    def forward(self,X) :
        Z = self.SoftMax.forward(X)
        Y = self.CrossEntropy.forward(Z)
        return Y
    def backward(self,dY) :
        dZ = self.CrossEntropy.backward(dY)
        dX = self.SoftMax.backward(dY,self.CrossEntropy.T)


In [34]:
class MutiClassClassification :
    def __init__(self,inputsize,outputsize) :
        W1, b1 = init_Xavier_weights(inputsize,256)
        self.Linear1 = Linear(W1,b1)
        self.ReLU = ReLU()
        W2,b2 = init_Xavier_weights(256,outputsize)
        self.Linear2 = Linear(W2,b2)
        self.Softmax = SoftMax()
    
    def forward(self,X) :
        x1 = self.Linear1(X)
        x2 = self.ReLU(x2)
        x3 = self.Linear2(x2)
        self.x3 = x3
        Y = self.Softmax(x3)
        return Y
    
    def backward(self,dY) :
        dX2,dW2,db2 = self.Linear2.backward(dY)
        dZ1 = self.ReLU.backward(dX2)
        dX1, dW1, db1 = self.Linear1.backward(dZ1)
        return dX1,dW1,db1,dZ1,dX2,dW2,db2
